In [1]:
import sys

sys.path.append("../")  # up one level

import os
import shutil

import gcsfs
import geopandas as gpd
import pandas as pd
import annual_ridership_module

from calitp_data_analysis.sql import to_snakecase
from calitp_data_analysis.tables import tbls
from segment_speed_utils.project_vars import PUBLIC_GCS
from siuba import _, collect, count, filter, select, show_query
from update_vars import GCS_FILE_PATH, NTD_MODES, NTD_TOS

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/ntd/"

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Work plan

## COMPLETE ~~Need List of all annual reporters~~
- Full, Reduced, Rural reporters all submit an annual Report to NTD
- used `dim_annual_service_agencies`

## COMPLETE ~~Need to assign rural and reduced annual reporters to RTPAs~~ 
- Full reporters are already mapped to RTPAs via the monthly report
- but reduced and rurap reporters need to be mapped to RTPAs
- sjoined a map of CA census designated places to RTPA map, then merged with `dim_annual_service_agenices` to attach RTPA info

## COMPLETE ~~Export processed data to parquet~~
- to be used for aggregation

## COMPLTE ~~Make new ntd_id to RTPA crosswalk~~
- to be used for future analyses and merges

## COMPLETE ~~Import formulas from monthly report for aggregation~~
- modified `produce_annual_ntd_ridership_data_by_rtpa`

## Test using data from new warehouse table `fct_service_data_and_operating_expenses_time_series_by_mode_upt`
- https://dbt-docs.calitp.org/#!/model/model.calitp_warehouse.fct_service_data_and_operating_expenses_time_series_by_mode_upt
---

In [22]:
## 

confirm_crosswalk = ntd_service.merge(
    just_ntd_rtpa_info,
    how="left",
    on="ntd_id",
    indicator=True
)

display(
    confirm_crosswalk.info(),
    confirm_crosswalk["_merge"].value_counts()
)
#GTG

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421 entries, 0 to 420
Data columns (total 16 columns):
 #   Column                                       Non-Null Count  Dtype   
---  ------                                       --------------  -----   
 0   report_year                                  421 non-null    int64   
 1   ntd_id                                       421 non-null    object  
 2   agency_x                                     421 non-null    object  
 3   reporter_type_x                              421 non-null    object  
 4   organization_type                            421 non-null    object  
 5   city_x                                       421 non-null    object  
 6   state                                        421 non-null    object  
 7   primary_uza_name                             328 non-null    object  
 8   actual_vehicles_passenger_car_revenue_hours  421 non-null    float64 
 9   actual_vehicles_passenger_car_revenue_miles  421 non-null    floa

None

both          421
left_only       0
right_only      0
Name: _merge, dtype: int64

## read in `fct_service_data_and_operating_expenses_time_series_by_mode_upt`

In [1]:
# new warehouse table
# includes upt, mode, tos and report year starting from 2018
# contains historical,inactive agencies

get_ntd_time_series = (
    tbls.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_upt()
    >> filter(_.state.str.contains("CA") | 
              _.state.str.contains("NV"), # to get lake Tahoe Transportation back
              _.year >= "2018",
              _.city != None,
              _.primary_uza_name.str.contains(", CA") | 
              _.primary_uza_name.str.contains("CA-NV") |
              _.primary_uza_name.str.contains("California Non-UZA") | 
              _.primary_uza_name.str.contains("El Paso, TX--NM") # something about Paso 
             )
    >> select(
        'agency_name',
        'agency_status',
        'city',
        'legacy_ntd_id',
        'mode',
        'ntd_id',
        'reporter_type',
        'reporting_module',
        'service',
        'state',
        'uace_code',
        'primary_uza_name',
        'uza_population',
        'year',
        'upt',
    )
    >> collect()
)

#what does group by look like
ntd_time_series = get_ntd_time_series.groupby(
    [
        "agency_name",
        'agency_status',
        "city",
        "state",
        "ntd_id",
        'primary_uza_name',
        "reporter_type",
        "mode",
        "service",
    ]
).agg({
    "upt":"sum"
}).sort_values(by="ntd_id").reset_index()

display(
    get_ntd_time_series.info(),
    ntd_time_series.info(),
    ntd_time_series["state"].value_counts(),
    ntd_time_series["primary_uza_name"].value_counts()
    
)

NameError: name 'tbls' is not defined

In [69]:
ntd_time_series[~ntd_time_series["primary_uza_name"].str.contains(", CA")]["primary_uza_name"].value_counts()
# who has uza in texas new mex?!

California Non-UZA    1020
El Paso, TX--NM         12
Name: primary_uza_name, dtype: int64

In [70]:
ntd_time_series[ntd_time_series["primary_uza_name"].str.contains("El Paso, TX--NM")] # just 1 agency, PAso Robles Transit Services

,agency_name,agency_status,city,legacy_ntd_id,mode,ntd_id,reporter_type,reporting_module,service,state,uace_code,primary_uza_name,uza_population,year,upt
1140,Paso Robles Transit Services (PE),Inactive,Paso Robles,9195,DR,90195,Reduced Reporter,None,PT,CA,27253,"El Paso, TX--NM",854584,2018,NaN
1141,Paso Robles Transit Services (PE),Inactive,Paso Robles,9195,DR,90195,Reduced Reporter,None,PT,CA,27253,"El Paso, TX--NM",854584,2022,NaN
1142,Paso Robles Transit Services (PE),Inactive,Paso Robles,9195,DR,90195,Reduced Reporter,None,PT,CA,27253,"El Paso, TX--NM",854584,2021,NaN
1143,Paso Robles Transit Services (PE),Inactive,Paso Robles,9195,MB,90195,Reduced Reporter,None,PT,CA,27253,"El Paso, TX--NM",854584,2022,NaN
1144,Paso Robles Transit Services (PE),Inactive,Paso Robles,9195,DR,90195,Reduced Reporter,None,PT,CA,27253,"El Paso, TX--NM",854584,2020,NaN
1145,Paso Robles Transit Services (PE),Inactive,Paso Robles,9195,DR,90195,Reduced Reporter,None,PT,CA,27253,"El Paso, TX--NM",854584,2019,NaN
1146,Paso Robles Transit Services (PE),Inactive,Paso Robles,9195,MB,90195,Reduced Reporter,None,PT,CA,27253,"El Paso, TX--NM",854584,2019,NaN
1147,Paso Robles Transit Services (PE),Inactive,Paso Robles,9195,MB,90195,Reduced Reporter,None,PT,CA,27253,"El Paso, TX--NM",854584,2020,NaN
1148,Paso Robles Transit Services (PE),Inactive,Paso Robles,9195,MB,90195,Reduced Reporter,None,PT,CA,27253,"El Paso, TX--NM",854584,2021,NaN
1149,Paso Robles Transit Services (PE),Inactive,Paso Robles,9195,MB,90195,Reduced Reporter,None,PT,CA,27253,"El Paso, TX--NM",854584,2018,NaN


In [83]:
# how many cities in ntd_time_serires are there?
# wonder if all the cities will be in the crosswalk
ntd_time_series["city"].nunique() # 183. should be enough


183

## See `explore_rtpa_ntd_crosswalk` notebook to see how the new rtpa to ntd id crosswalk was made

## read in crosswalk from gcs

In [84]:
test_data = pd.read_parquet(f"{GCS_FILE_PATH}annual_report_data_2022-2023.parquet")
rtpa_ntd_crosswalk = pd.read_parquet(f"{GCS_FILE_PATH}ntd_id_rtpa_crosswalk_all_reporter_types.parquet")

display(
    test_data.head(),
    rtpa_ntd_crosswalk.head()
)

,report_year,ntd_id,agency,reporter_type,organization_type,city,state,primary_uza_name,actual_vehicles_passenger_car_revenue_hours,actual_vehicles_passenger_car_revenue_miles,unlinked_passenger_trips_upt,cdp_name,RTPA,_merge
0,2022,90003,San Francisco Bay Area Rapid Transit District,Full Reporter,Independent Public Agency or Authority of Tran...,Oakland,CA,"San Francisco--Oakland, CA",2486029.0,78554914.0,38224072.0,Oakland,Metropolitan Transportation Commission,both
1,2023,90003,"San Francisco Bay Area Rapid Transit District,...",Full Reporter,Independent Public Agency or Authority of Tran...,Oakland,CA,"San Francisco--Oakland, CA",2724074.0,85233749.0,50764402.0,Oakland,Metropolitan Transportation Commission,both
2,2022,90004,Golden Empire Transit District,Full Reporter,Independent Public Agency or Authority of Tran...,Bakersfield,CA,"Bakersfield, CA",292325.0,3758507.0,3201046.0,Bakersfield,Kern Council of Governments,both
3,2023,90004,Golden Empire Transit District,Full Reporter,Independent Public Agency or Authority of Tran...,Bakersfield,CA,"Bakersfield, CA",289338.0,3924016.0,3293593.0,Bakersfield,Kern Council of Governments,both
4,2023,90006,Santa Cruz Metropolitan Transit District,Full Reporter,Independent Public Agency or Authority of Tran...,Santa Cruz,CA,"Santa Cruz, CA",214748.0,2975126.0,3350026.0,Santa Cruz,Santa Cruz County Regional Transportation Comm...,both


,ntd_id,agency,reporter_type,city,RTPA
0,90003,"San Francisco Bay Area Rapid Transit District,...",Full Reporter,Oakland,Metropolitan Transportation Commission
1,90004,Golden Empire Transit District,Full Reporter,Bakersfield,Kern Council of Governments
2,90006,Santa Cruz Metropolitan Transit District,Full Reporter,Santa Cruz,Santa Cruz County Regional Transportation Comm...
3,90008,"City of Santa Monica, dba: Big Blue Bus",Full Reporter,Santa Monica,Southern California Association of Governments
4,90009,San Mateo County Transit District,Full Reporter,San Carlos,Metropolitan Transportation Commission


## test merge of `ntd_time_series` with `rtpa ntd crosswalk`

In [89]:
ntd_time_series_rtpa = ntd_time_series.merge(
    rtpa_ntd_crosswalk,
    on="ntd_id",
    how="left",
    indicator=True
)
display(
    ntd_time_series_rtpa.info(),
    ntd_time_series_rtpa["_merge"].value_counts()
)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4152 entries, 0 to 4151
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   agency_name       4152 non-null   object  
 1   agency_status     4152 non-null   object  
 2   city_x            4008 non-null   object  
 3   legacy_ntd_id     3426 non-null   object  
 4   mode              4152 non-null   object  
 5   ntd_id            4002 non-null   object  
 6   reporter_type_x   4152 non-null   object  
 7   reporting_module  3726 non-null   object  
 8   service           4152 non-null   object  
 9   state             4152 non-null   object  
 10  uace_code         4152 non-null   int64   
 11  primary_uza_name  4152 non-null   object  
 12  uza_population    4152 non-null   int64   
 13  year              4152 non-null   object  
 14  upt               2623 non-null   float64 
 15  agency            3552 non-null   object  
 16  reporter_type_y   3552 n

None

both          3552
left_only      600
right_only       0
Name: _merge, dtype: int64

In [94]:
# what agencies or cities did not merge?
# how many of these agencies are inactive status?

display(
    ntd_time_series_rtpa[ntd_time_series_rtpa["_merge"]=="left_only"][["agency_status","agency_name"]].value_counts(),
    ntd_time_series_rtpa[ntd_time_series_rtpa["_merge"]=="left_only"]["agency_name"].unique(),
    ntd_time_series_rtpa[ntd_time_series_rtpa["_merge"]=="left_only"]["city_x"].unique()
)

agency_status  agency_name                                                      
Inactive       City of Dinuba                                                       30
               City of Modesto (MAX)                                                30
               City of Lincoln (COL)                                                24
               El Dorado County Transit Authority                                   24
               Los Angeles County Metropolitan Transportation Authority (LACMTA)    18
               City of Elk Grove(etran)                                             18
               LACMTA - Small Operators (LACMTA)                                    18
               City of Vallejo Transportation Program (Vallejo Transit, Baylink)    18
               Susanville Indian Rancheria(SIR)                                     18
               Stanislaus County (StaRT)                                            18
               Los Angeles County Transportation 

array(['ATC / Vancom', 'Aero Airport Shuttle and Charter',
       'Calaveras County Department of Public Works',
       'California Dept of Transportation San Diegan Commuter Rail',
       'Chico Area Transit System City of Chico (CATS)',
       'Chula Vista Transit (CVT)', 'City of Alameda Ferry Services',
       'City of Barstow', 'City of Benicia (Benicia Breeze)',
       'City of Dinuba', 'City of Duarte', 'City of Elk Grove(etran)',
       'City of Folsom (FSL)', 'City of Irvine (COI)', 'City of Lawndale',
       'City of Lincoln (COL)', 'City of Merced Transit System',
       'City of Modesto (MAX)', 'City of South Lake Tahoe',
       'City of Tulare(TIME)',
       'City of Vallejo Transportation Program (Vallejo Transit, Baylink)',
       'City of Woodlake(MCC/ESCORT)',
       'County of San Diego Transit System (CTS)',
       'County of Shasta Department of Public Works',
       'DAVE Transportation Services, Inc.', 'Duckwater Shoshone Tribe',
       'El Dorado County Departmen

array(['Oakland', None, 'San Andreas', 'Sacramento', 'Chico',
       'Chula Vista', 'Alameda', 'Benicia', 'Dinuba', 'Duarte',
       'Elk Grove', 'Folsom', 'Irvine', 'Lawndale', 'Lincoln', 'Merced',
       'Modesto', 'Tulare', 'Vallejo', 'Woodlake', 'San Diego', 'Redding',
       'Sherman Oaks', 'El Centro', 'Los Angeles', 'El Monte', 'San Jose',
       'Salinas', 'National City', 'Paso Robles', 'Redondo Beach',
       'West Covina', 'San Francisco', 'Rosemead', 'San Luis Obispo',
       'Susanville'], dtype=object)

## Full Test `produce_annual_ntd_ridership_data_by_rtpa` function

In [78]:
def produce_annual_ntd_ridership_data_by_rtpa():
    """
    Function that ingest ridership data from `dim_annual_service_agencies`, filters for CA agencies.
    Merges in ntd_id_to_RTPA_crosswalk.
    
    """
    from annual_ridership_module import add_change_columns
    
    print("ingest annual ridership data from warehouse")
    
    ntd_service = (
    tbls.mart_ntd.dim_annual_service_agencies()
    >> filter(_.state == "CA")
    >> select(
        "report_year",
        "ntd_id",
        "agency",
        "reporter_type",
        "organization_type",
        "city",
        "state",
        "primary_uza_name",
        "actual_vehicles_passenger_car_revenue_hours",
        "actual_vehicles_passenger_car_revenue_miles",
        "unlinked_passenger_trips_upt",
    )
    >> collect())
        
    print("clean up warehouse data")
    
    update_dict={
        "90227":"Moorpark",
        "90253":"Bell Gardens",
        "90259":"Cerritos",
        "90286":"Monterey Park",
    }
    
    for i, v in update_dict.items():
        ntd_service.loc[ntd_service["ntd_id"]== i,"city"] = v
    
    print("read in new `ntd_id_to_rtpa_all_reporter_types` crosswalk") 
    
    ntd_to_rtpa_crosswalk = pd.read_parquet(f"{GCS_FILE_PATH}ntd_id_rtpa_crosswalk_all_reporter_types.parquet")
        
    print("merge ntd data to crosswalk")
    
    ntd_data_by_rtpa = ntd_service.merge(
    ntd_to_rtpa_crosswalk,
    how="left",
    on=[
        "ntd_id",
        #"agency", # sometime agency name change, but ntd id stays the same. causing unmerged rows
        #"reporter_type",
        #"city"
    ],
    indicator=True
    ).rename(
    columns={
       "actual_vehicles_passenger_car_revenue_hours":"vrh",
       "actual_vehicles_passenger_car_revenue_miles":"vrm",
       "unlinked_passenger_trips_upt":"upt" 
    }
    )
    
    print(ntd_data_by_rtpa._merge.value_counts())
        
    if len(ntd_data_by_rtpa[ntd_data_by_rtpa._merge=="left_only"]) > 0:
        raise ValueError("There are unmerged rows to crosswalk")
    
    print("add `change_column` to data")
    ntd_data_by_rtpa = add_change_columns(ntd_data_by_rtpa)
    
    return ntd_data_by_rtpa

# will add step to export data to gcs in final script

In [79]:
annual_ridership_report_data = produce_annual_ntd_ridership_data_by_rtpa()

ingest annual ridership data from warehouse
clean up warehouse data
read in new `ntd_id_to_rtpa_all_reporter_types` crosswalk
merge ntd data to crosswalk
both          421
left_only       0
right_only      0
Name: _merge, dtype: int64
add `change_column` to data


/home/jovyan/data-analyses/ntd/annual_ridership_report/annual_ridership_module.py:44: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda x: x.shift(1))


### export report data to GCS, as parquet,
- to be read in by portfolio

In [27]:
# annual_ridership_report_data.to_parquet(f"{GCS_FILE_PATH}annual_ridership_report_data.parquet")

## read in data from GCS to make sure things work 

In [28]:
annual_ridership_report_data= pd.read_parquet(f"{GCS_FILE_PATH}annual_ridership_report_data.parquet")

In [48]:
annual_ridership_report_data['RTPA'].value_counts()

Southern California Association of Governments          194
Metropolitan Transportation Commission                   50
Kern Council of Governments                              21
San Joaquin Council of Governments                       14
Sacramento Area Council of Governments                   14
Santa Barbara County Association of Governments          14
Humboldt County Association of Governments                8
Madera County Transportation Commission                   8
Placer County Transportation Planning Agency              6
San Diego Association of Governments                      6
Tulare County Association of Governments                  6
San Luis Obispo Council of Governments                    6
Kings County Association of Governments                   6
Stanislaus Council of Governments                         5
Del Norte Local Transportation Commission                 5
Merced County Association of Governments                  4
Fresno Council of Governments           